In [1]:
import pandas as pd
import numpy as np 
#!pip install pyarrow

In [2]:
jan_df=pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")

In [3]:
jan_df.shape

(3066766, 19)

In [4]:
jan_df.head(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
5,2,2023-01-01 00:50:34,2023-01-01 01:02:52,1.0,1.84,1.0,N,161,137,1,12.8,1.00,0.5,10.00,0.0,1.0,27.80,2.5,0.00
6,2,2023-01-01 00:09:22,2023-01-01 00:19:49,1.0,1.66,1.0,N,239,143,1,12.1,1.00,0.5,3.42,0.0,1.0,20.52,2.5,0.00
7,2,2023-01-01 00:27:12,2023-01-01 00:49:56,1.0,11.70,1.0,N,142,200,1,45.7,1.00,0.5,10.74,3.0,1.0,64.44,2.5,0.00
8,2,2023-01-01 00:21:44,2023-01-01 00:36:40,1.0,2.95,1.0,N,164,236,1,17.7,1.00,0.5,5.68,0.0,1.0,28.38,2.5,0.00
9,2,2023-01-01 00:39:42,2023-01-01 00:50:36,1.0,3.01,1.0,N,141,107,2,14.9,1.00,0.5,0.00,0.0,1.0,19.90,2.5,0.00


In [5]:
#Now let's compute the duration variable. It should contain the duration of a ride in minutes.
#What's the standard deviation of the trips duration in January?

jan_df['duration'] = (jan_df['tpep_dropoff_datetime'] - jan_df['tpep_pickup_datetime']).dt.total_seconds() / 60
jan_df['duration'].std()

42.594351241920904

In [6]:
"""Next, we need to check the distribution of the duration variable. \
There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

"""
import matplotlib.pyplot as plt
import seaborn as sns 
jan_df.query('(duration>=1) & (duration<=60)').shape

(3009173, 20)

In [7]:
(3009173-3066766)/3066766*100

-1.8779717787402104

In [8]:
"""Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it"""

"Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.\n\nTurn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)\nFit a dictionary vectorizer\nGet a feature matrix from it"

In [22]:
#one hot encode these 
jan_df=jan_df.query('(duration>=1) & (duration<=60)')
X=jan_df[['PULocationID', 'DOLocationID']].astype(str)
X=X.to_dict(orient='records')
#X=dict(zip(jan_df['PULocationID'], jan_df['DOLocationID']))
X

[{'PULocationID': '161', 'DOLocationID': '141'},
 {'PULocationID': '43', 'DOLocationID': '237'},
 {'PULocationID': '48', 'DOLocationID': '238'},
 {'PULocationID': '138', 'DOLocationID': '7'},
 {'PULocationID': '107', 'DOLocationID': '79'},
 {'PULocationID': '161', 'DOLocationID': '137'},
 {'PULocationID': '239', 'DOLocationID': '143'},
 {'PULocationID': '142', 'DOLocationID': '200'},
 {'PULocationID': '164', 'DOLocationID': '236'},
 {'PULocationID': '141', 'DOLocationID': '107'},
 {'PULocationID': '234', 'DOLocationID': '68'},
 {'PULocationID': '79', 'DOLocationID': '264'},
 {'PULocationID': '164', 'DOLocationID': '143'},
 {'PULocationID': '138', 'DOLocationID': '33'},
 {'PULocationID': '33', 'DOLocationID': '61'},
 {'PULocationID': '79', 'DOLocationID': '186'},
 {'PULocationID': '90', 'DOLocationID': '48'},
 {'PULocationID': '113', 'DOLocationID': '255'},
 {'PULocationID': '237', 'DOLocationID': '239'},
 {'PULocationID': '143', 'DOLocationID': '229'},
 {'PULocationID': '137', 'DOLocat

In [23]:
#Fit a dictionary vectorizer
from sklearn.feature_extraction import DictVectorizer

vectorizer = DictVectorizer()
one_hot_encoded = vectorizer.fit_transform(X)

#get number of columns
num_features = one_hot_encoded.shape[1]
print(num_features)

515


In [16]:
#CREATE A LINEAR MODEL 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [24]:
y=jan_df['duration']
X_train,X_test,y_train,y_test=train_test_split(one_hot_encoded,y,test_size=0.2)

In [25]:
model=LinearRegression()
model.fit(X_train,y_train)
y_train_pred=model.predict(X_train)
from sklearn.metrics import mean_squared_error
np.sqrt(mean_squared_error(y_train,y_train_pred))

7.6475920086989

In [27]:
np.sqrt(mean_squared_error(y_test,model.predict(X_test)))

7.658709283317507